In [1]:
import pandas as pd
import numpy as np
import csv
import os
import time

In [2]:
from gp_pref_elicit_luisa import gaussian_process as GP 
from gp_pref_elicit_luisa import dataset as data 
from gp_pref_elicit_luisa import acquisition_function as acquisition_function
from gp_pref_elicit_luisa.gp_utilities import utils_ccs as gp_utils_ccs
from gp_pref_elicit_luisa.gp_utilities import utils_data as gp_utils_data
from gp_pref_elicit_luisa.gp_utilities import utils_user as gp_utils_users

In [3]:
# generating synthetic dataset which outputs value vectors
# to generate new dataset: uncomment the line below and comment the dataset above
# gp_utils_ccs.get_ccs(num_objectives=2, ccs_size=100) #feel free to modify the number of objectives
# note: after getting the synthetic data, store the data in a variable and comment the line above otherwise this will 
# not give proper insights for experiments as the data will be generated again.
# synthetic_pcs
# outputs a synthetic Pareto Coverage Set of value vectors with 2 objectives and 100 datapoints synthetic Pareto Coverage Set of value vectors with 2 objectives and 20 datapoints

In [4]:
# 5 obj with 20 datapoints
synthetic_pcs = np.array([[0.83322774, 0.00580742, 0.33171299, 0.15411129, 0.40810134],
       [0.36035758, 0.31087373, 0.87092702, 0.15053075, 0.10724224],
       [0.4469999 , 0.70892228, 0.19793838, 0.46892235, 0.13679864],
       [0.30406647, 0.60188963, 0.18040018, 0.41098283, 0.57872045],
       [0.11978041, 0.09624517, 0.81627444, 0.01526226, 0.55035718],
       [0.48061058, 0.54886427, 0.16476036, 0.62658487, 0.17486296],
       [0.68429025, 0.12493305, 0.63153446, 0.29202693, 0.04903378],
       [0.39680506, 0.83823222, 0.05121777, 0.19420762, 0.31756707],
       [0.15227467, 0.77752572, 0.62661162, 0.15790781, 0.02792035],
       [0.26091085, 0.68211945, 0.51244374, 0.16934385, 0.47931674],
       [0.45643899, 0.73087359, 0.23199633, 0.16495828, 0.41870565],
       [0.07773575, 0.76072589, 0.34722822, 0.5112436 , 0.00317963],
       [0.09559396, 0.03664719, 0.02798014, 0.53256982, 0.84261984],
       [0.7706333 , 0.14344632, 0.58580674, 0.20141788, 0.14233396],
       [0.07470304, 0.23418028, 0.57539888, 0.23529168, 0.70879152],
       [0.23109497, 0.29060082, 0.78178992, 0.27394011, 0.44124146],
       [0.1770374 , 0.18222337, 0.96587018, 0.0803865 , 0.00635683],
       [0.07667673, 0.85270688, 0.24718199, 0.06201355, 0.44649782],
       [0.57582347, 0.01073251, 0.52007739, 0.65108527, 0.07253401],
       [0.14388769, 0.03662216, 0.61677147, 0.78382862, 0.0917281 ]])

In [5]:
# initializing Gaussian Process, Dataset
GP = GP.GPPairwise(num_objectives=5)
utils_comparisons = data.DatasetPairwise(num_objectives=5)
acquisition_function_DA = acquisition_function.DiscreteAcquirer(input_domain=synthetic_pcs, query_type='pairwise', seed=None, acquisition_type='expected improvement')
acquisition_function_EI = acquisition_function.get_expected_improvement(datapoints=synthetic_pcs, gaussian_process=GP, datapoints_hist=acquisition_function_DA.history, xi=0.01)
# getting user preferences to generate a ground truth utility function
user_pref = gp_utils_users.UserPreference(num_objectives=5, std_noise=0.1)

In [6]:
# ground truth utility function for synthetic dataset
ground_truth_utility_function_dataset = user_pref.get_preference(synthetic_pcs, add_noise=True)

# getting the maximum utility which is the actual best utility
ground_truth_utility_function_dataset = np.max(ground_truth_utility_function_dataset)
ground_truth_utility_function_dataset

0.6365722130089095

In [7]:
# starting points in the dataset
start_points = acquisition_function_DA.get_start_points(gaussian_process=GP) 

# generating ground truth utility function for the starting points
ground_truth_utility_function_start = user_pref.get_preference(start_points, add_noise=True)

# getting the index of the highest utility
highest_utility_index_start = np.argmax(ground_truth_utility_function_start)

# getting the index of the lowest utility
lowest_utility_index_start = np.argmin(ground_truth_utility_function_start)

# mapping the indices of the highest utility to the actual datapoint in the dataset
highest_utility_point_dataset = start_points[highest_utility_index_start]

# mapping the indices of the lowest utility to the actual datapoint in the dataset
lowest_utility_point_dataset = start_points[lowest_utility_index_start]


# adding the highest utility point, which is the winner, to the dataset 
utils_comparisons.add_single_comparison(highest_utility_point_dataset, lowest_utility_point_dataset)

# adding the actual point to the array 
current_best_point = highest_utility_point_dataset

# updating the GP
GP.update(utils_comparisons)

# checking which points to exclude
exclude_points = acquisition_function.exclude_points_pairwise(dataset=utils_comparisons)

print('Start points: ', start_points, '\n',
      'Point in the dataset having highest utility: ', highest_utility_point_dataset, '\n',
      'Point in the dataset having lowest utility: ', lowest_utility_point_dataset, '\n',
      'Current max: ', current_best_point, '\n', 
      'Excluded points: ', exclude_points)

Start points:  (array([0.68429025, 0.12493305, 0.63153446, 0.29202693, 0.04903378]), array([0.23109497, 0.29060082, 0.78178992, 0.27394011, 0.44124146])) 
 Point in the dataset having highest utility:  [0.68429025 0.12493305 0.63153446 0.29202693 0.04903378] 
 Point in the dataset having lowest utility:  [0.23109497 0.29060082 0.78178992 0.27394011 0.44124146] 
 Current max:  [0.68429025 0.12493305 0.63153446 0.29202693 0.04903378] 
 Excluded points:  [[0.68429025 0.12493305 0.63153446 0.29202693 0.04903378]
 [0.23109497 0.29060082 0.78178992 0.27394011 0.44124146]]


We have 2 stopping conditions (similar to the GP with EI):
1. If the next point found according to Expected Improvement is already in the list of compared points (the points which we exclude for comparison - excluded points) then the loop breaks. This is because the loop has explored all the points available to it.
2. If we have a large dataset where the above condition might not always be true then we look at the probability of improvement between points. If the probability of improvement is below 5% (0.05) then the loop terminates because there is no further scope of improvement.

In [8]:
# setting the threshold to 5%
threshold = 0.05
stop = False
counter = 0 # counter variable to keep track of number of queries 
last_counter = 0 # to store the total number of queries (which we get from the last iteration)
last_regret = 0 # to store the regret gained from the last iteration

first_run = not os.path.exists('experiments/output-TS_5-20.csv') # checking if the file exists already in the directory

starting_time = time.time() # starting the timer

while not stop:
  # next points in the dataset according to Thompson Sampling 
  next_point_TS = acquisition_function_DA.get_next_point_thompson(gaussian_process=GP, exclude=exclude_points) 

  # applying first stopping condition
  if next_point_TS.tolist() in exclude_points.tolist():
    last_counter = counter
    last_regret = regret
    stop = True


  # generating ground truth utility function for the next points
  ground_truth_utility_function_next_TS = user_pref.get_preference(next_point_TS, add_noise=True)
  # generating ground truth utility function for the current best point
  ground_truth_utility_function_current_best = user_pref.get_preference(current_best_point, add_noise=True)

  # adding the highest utility point, which is the winner, to the dataset only if the current point being evaluated according to 
  # expected improvement has a greater utility than the utility of the current best point evaluated previously
  if ground_truth_utility_function_next_TS > ground_truth_utility_function_current_best:
    utils_comparisons.add_single_comparison(next_point_TS, current_best_point)
    current_best_point = next_point_TS # we update the current best point 
  else:
    utils_comparisons.add_single_comparison(current_best_point, next_point_TS)
  
  # computing the datapoints for this comparisons dataset
  utility_comparisons_datapoints_next_EI = utils_comparisons.datapoints

  # updating the GP
  GP.update(utils_comparisons)

  # checking which points to exclude
  exclude_points = acquisition_function.exclude_points_pairwise(dataset=utils_comparisons)
  exclude_points = np.append(exclude_points, [current_best_point], axis=0)

  # difference between actual best utility and observed best utility
  regret = np.subtract(ground_truth_utility_function_dataset, ground_truth_utility_function_current_best)

  # calculating the probability of improvement
  prob_imprv = acquisition_function.get_probability_of_improvement(x=next_point_TS, gaussian_process=GP, x_previous=current_best_point)

  # incrementing the counter variable
  counter +=1

  # stop the timer
  ending_time = time.time()
  total_time = ending_time - starting_time

  print('Excluded points: ', exclude_points, '\n',
    'next points for the GP accordint to Thompson Sampling: ', next_point_TS, '\n',
    'Comparison dataset datapoints: ', utility_comparisons_datapoints_next_EI, '\n',
    'Current max: ', current_best_point, '\n', 
    'Ground truth utility of the current best: ', ground_truth_utility_function_current_best, '\n',
    'Excluded points: ', len(exclude_points), '\n',
    'Simple Regret: ', regret, '\n',
    'Probability of improvement: ', prob_imprv, '\n',
    'Number of queries: ', counter, '\n',
    'Computation Time: ', total_time)
  
  # applying second stopping condition
  if prob_imprv < threshold:
    last_counter = counter
    last_regret = regret
    stop = True

# we output the counter and regret of the last iteration to a csv for comparisons
with open('experiments/output-TS_5-20.csv', 'a', newline='') as csvfile:
  write_to_csv = csv.writer(csvfile)

  # write the header row only for the first run
  if first_run: # if the file does not exist then we output the header otherwise we output only the data
    write_to_csv.writerow(['Number of Queries', 'Regret', 'Computation Time'])
  write_to_csv.writerow([last_counter + 1, last_regret, total_time])
print('Successfully written to csv')


Excluded points:  [[0.26091085 0.68211945 0.51244374 0.16934385 0.47931674]
 [0.68429025 0.12493305 0.63153446 0.29202693 0.04903378]
 [0.26091085 0.68211945 0.51244374 0.16934385 0.47931674]] 
 next points for the GP accordint to Thompson Sampling:  [0.26091085 0.68211945 0.51244374 0.16934385 0.47931674] 
 Comparison dataset datapoints:  [[0.68429025 0.12493305 0.63153446 0.29202693 0.04903378]
 [0.23109497 0.29060082 0.78178992 0.27394011 0.44124146]
 [0.26091085 0.68211945 0.51244374 0.16934385 0.47931674]] 
 Current max:  [0.26091085 0.68211945 0.51244374 0.16934385 0.47931674] 
 Ground truth utility of the current best:  [0.23425611] 
 Excluded points:  3 
 Simple Regret:  [0.4023161] 
 Probability of improvement:  [0.48360452] 
 Number of queries:  1 
 Computation Time:  0.02152419090270996
Excluded points:  [[0.48061058 0.54886427 0.16476036 0.62658487 0.17486296]
 [0.26091085 0.68211945 0.51244374 0.16934385 0.47931674]
 [0.48061058 0.54886427 0.16476036 0.62658487 0.17486296]